In [ ]:
import pandas as pd
import os

# --- 1. Definição dos Caminhos e Nomes de Colunas ---

# Caminho para a pasta onde o arquivo está localizado
caminho_pasta = '../data/raw'

# Nome do arquivo CSV
nome_arquivo = 'Cópia de Histeroscopia Diagnóstica (respostas) 2023 - Respostas ao formulário 1.csv'

# Construção do caminho completo do arquivo
caminho_completo = os.path.join(caminho_pasta, nome_arquivo)

# Lista com o nome exato das colunas que devem ser carregadas
colunas_para_carregar = [
    "Idade",
    "Indique sua condição de saúde atual",
    "Conte um pouco sobre o motivo para solicitar a consulta. ",
    "Número de prontuário no CISAM\nVocê só poderá continuar este preenchimento se tiver prontuário, caso vc não tenha prontuário no CISAM, clique aqui."
]

# --- 2. Carregamento do Arquivo CSV ---

try:
    # Carrega o arquivo CSV especificando apenas as colunas desejadas
    df = pd.read_csv(caminho_completo, usecols=colunas_para_carregar)

    # --- NOVA SEÇÃO: 3. Renomeando as Colunas ---

    # Dicionário para mapear os nomes antigos para os novos
    # 'nome antigo': 'nome novo'
    novos_nomes = {
        "Idade": "idade",
        "Indique sua condição de saúde atual": "condicao_saude",
        "Conte um pouco sobre o motivo para solicitar a consulta. ": "motivo_consulta",
        "Número de prontuário no CISAM\nVocê só poderá continuar este preenchimento se tiver prontuário, caso vc não tenha prontuário no CISAM, clique aqui.": "prontuario_cisam"
    }

    # A função rename é chamada no dataframe 'df'.
    # O parâmetro 'columns' recebe o dicionário com os novos nomes.
    # 'inplace=True' modifica o dataframe diretamente, sem precisar criar um novo.
    df.rename(columns=novos_nomes, inplace=True)

    # --- 4. Exibição dos Dados com as Colunas Renomeadas ---

    print("Dados carregados e colunas renomeadas com sucesso!")
    print("Visualizando as 5 primeiras linhas com os novos nomes de colunas:")
    display(df.head())

    print("\nInformações sobre o DataFrame (note os novos nomes nas colunas):")
    df.info()

except FileNotFoundError:
    print(f"Erro: O arquivo não foi encontrado no caminho especificado:\n{caminho_completo}")
    print("\nPor favor, verifique se o nome do arquivo e o caminho para a pasta '../data/raw' estão corretos.")
except ValueError as e:
    print(f"Erro ao carregar as colunas. Verifique se os nomes das colunas em 'colunas_para_carregar' estão exatamente como no arquivo CSV.")
    print(f"Detalhe do erro: {e}")

In [ ]:
# --- Garante que o DataFrame 'df' existe antes de iniciar ---
if 'df' in locals():

    # --- 0. Ponto de Partida: Análise inicial ---
    print("--- ANÁLISE INICIAL DO DATAFRAME 'df' ---")
    print(f"Número total de linhas (registros) antes da operação: {len(df)}")
    print(f"Número de prontuários únicos: {df['prontuario_cisam'].nunique()}")
    print("-" * 50)


    # --- 1. Remover todos os dados nulos da coluna 'prontuario_cisam' ---
    print("\n--- ETAPA 1: Removendo registros com prontuário nulo/vazio ---")
    df_clean = df.dropna(subset=['prontuario_cisam']).copy()

    # Garantia extra: remove também strings que contêm apenas espaços em branco
    # .astype(str) previne erros caso a coluna tenha tipos mistos
    df_clean = df_clean[df_clean['prontuario_cisam'].astype(str).str.strip() != '']
    print(f"Número de linhas após remover nulos: {len(df_clean)}")
    print("-" * 50)


    # --- 2. Unir valores repetidos com regras específicas ---
    print("\n--- ETAPA 2: Consolidando prontuários duplicados ---")
    print("Agrupando por 'prontuario_cisam' e aplicando regras de união...")

    # Define as regras de agregação para cada coluna
    regras_de_agregacao = {
        'idade': 'first',  # Pega o primeiro valor encontrado para a idade
        'condicao_saude': lambda s: ' | '.join(s.dropna().unique()), # Concatena valores únicos com ' | '
        'motivo_consulta': lambda s: ' | '.join(s.dropna().unique()) # Concatena valores únicos com ' | '
    }

    # Agrupa pelo prontuário e aplica as regras definidas
    # O .reset_index() transforma a coluna de agrupamento ('prontuario_cisam') de volta em uma coluna normal
    df_consolidado = df_clean.groupby('prontuario_cisam').agg(regras_de_agregacao).reset_index()

    print(f"Número de linhas após consolidar duplicados: {len(df_consolidado)}")
    print("Este número deve ser igual ao de prontuários únicos da análise inicial.")
    print("-" * 50)


    # --- 3. Verificar se sobraram valores repetidos ---
    print("\n--- ETAPA 3: Verificação final de duplicados ---")

    duplicados_restantes = df_consolidado['prontuario_cisam'].duplicated().any()

    if not duplicados_restantes:
        print("✅ SUCESSO! Não há mais nenhum valor duplicado na coluna 'prontuario_cisam'.")
    else:
        print("❌ ATENÇÃO! A operação falhou e ainda existem prontuários duplicados.")

    print("-" * 50)


    # --- 4. Exibição do Resultado Final ---
    print("\n--- RESULTADO FINAL ---")
    print("Visualizando as 5 primeiras linhas do DataFrame consolidado e limpo:")
    display(df_consolidado.head())

    # Verificando um exemplo de prontuário que pode ter sido concatenado (se houver duplicados)
    # print("\nExemplo de um registro que pode ter sido consolidado:")
    # display(df_consolidado[df_consolidado['motivo_consulta'].str.contains('\|', na=False)])


else:
    print("Erro: O DataFrame 'df' não foi encontrado. Por favor, execute a célula que o carrega primeiro.")

In [ ]:
# --- 1. Definição do Caminho e Colunas ---

# Nome do arquivo CSV
nome_arquivo_pacientes = 'Dados pacientes - Página1.csv'

# Construção do caminho completo para o arquivo
caminho_completo_pacientes = os.path.join(caminho_pasta, nome_arquivo_pacientes)

# --- Lista com as colunas específicas para carregar ---
colunas_pacientes_para_carregar = [
    "Prontuário",
    "Evolução"
]

# Dicionário para especificar que a coluna 'Prontuário' deve ser lida como texto (string)
tipos_dados = {
    'Prontuário': str
}


# --- 2. Carregamento do Arquivo CSV ---

try:
    # MODIFICADO: Adicionado o parâmetro 'dtype' para forçar o tipo da coluna
    df_pacientes = pd.read_csv(
        caminho_completo_pacientes, 
        usecols=colunas_pacientes_para_carregar,
        dtype=tipos_dados
    )

    # --- 3. Exibição dos Dados Carregados ---

    print("Arquivo de pacientes carregado com sucesso (apenas colunas selecionadas)!")
    print("Visualizando as 5 primeiras linhas:")
    display(df_pacientes.head())

    print("\nInformações sobre o DataFrame de pacientes (note as colunas carregadas):")
    # Agora, a saída de .info() mostrará 'Prontuário' como 'object' ou 'string', que é o tipo para texto no pandas.
    df_pacientes.info()

except FileNotFoundError:
    print(f"Erro: O arquivo não foi encontrado no caminho especificado:\n{caminho_completo_pacientes}")
    print("\nPor favor, verifique se o nome do arquivo está correto e se ele se encontra na pasta '../data/raw'.")
except ValueError as e:
    print(f"Erro ao carregar as colunas. Verifique se os nomes 'Prontuário' e 'Evolução' estão exatamente como no arquivo CSV.")
    print(f"Detalhe do erro: {e}")
except Exception as e:
    print(f"Ocorreu um erro inesperado ao tentar carregar o arquivo: {e}")

In [ ]:
# --- Garante que o DataFrame 'df_pacientes' existe antes de iniciar ---
if 'df_pacientes' in locals():

    print("--- Operando no DataFrame 'df_pacientes' ---")
    print(f"Número de linhas ANTES da limpeza: {len(df_pacientes)}")

    # --- 1. Remover valores vazios na coluna 'Prontuário' ---

    # Criamos uma cópia para não alterar o DataFrame original
    df_pacientes_limpo = df_pacientes.copy()

    # Etapa 1.1: Remove valores nulos (NaN)
    df_pacientes_limpo.dropna(subset=['Prontuário'], inplace=True)

    # Etapa 1.2: Como a coluna é String, removemos também strings vazias ou que contêm apenas espaços
    # O .astype(str) garante que o método .str funcione sem erros
    # O .str.strip() remove espaços em branco do início e do fim
    df_pacientes_limpo = df_pacientes_limpo[df_pacientes_limpo['Prontuário'].astype(str).str.strip() != '']

    print(f"Número de linhas APÓS a limpeza: {len(df_pacientes_limpo)}")
    print("-" * 50)


    # --- 2. Verificar se existem valores duplicados após a remoção ---
    print("\n--- Verificando duplicados na coluna 'Prontuário' limpa ---")

    # O método .value_counts() conta quantas vezes cada valor aparece
    contagem_prontuarios = df_pacientes_limpo['Prontuário'].value_counts()

    # Filtramos a contagem para encontrar apenas os prontuários que aparecem mais de uma vez
    prontuarios_duplicados = contagem_prontuarios[contagem_prontuarios > 1]

    # Verifica se a série de duplicados está vazia ou não
    if prontuarios_duplicados.empty:
        print("✅ SUCESSO! Nenhum valor duplicado foi encontrado na coluna 'Prontuário' após a limpeza.")
    else:
        print(f"❌ ATENÇÃO! Foram encontrados {len(prontuarios_duplicados)} prontuários duplicados após a limpeza.")
        print("Abaixo estão os prontuários e a quantidade de vezes que eles aparecem:")
        # A função display formata a saída de forma mais elegante no notebook
        display(prontuarios_duplicados)

    print("-" * 50)

else:
    print("Erro: O DataFrame 'df_pacientes' não foi encontrado. Por favor, execute a célula que o carrega primeiro.")

In [ ]:
# --- 1. Verificação dos DataFrames (Opcional, mas recomendado) ---
# Garante que os DataFrames df_consolidado e df_pacientes_limpo existem antes de prosseguir.
if 'df_consolidado' in locals() and 'df_pacientes_limpo' in locals():

    # --- 2. Junção dos DataFrames (Merge) ---
    # Usamos a função pd.merge para unir os dois DataFrames.
    # how='left': Mantém todas as linhas de 'df_consolidado' (o DataFrame da esquerda).
    # left_on='prontuario_cisam': Coluna chave do DataFrame da esquerda.
    # right_on='Prontuário': Coluna chave do DataFrame da direita.
    df_completo = pd.merge(df_consolidado, df_pacientes_limpo, how='left', left_on='prontuario_cisam', right_on='Prontuário')

    # --- 3. Exibição do Resultado ---
    print("DataFrames unidos com sucesso!")
    print("Visualizando as 5 primeiras linhas do DataFrame resultante:")
    # O display é mais adequado para notebooks, pois formata a tabela de forma mais legível.
    display(df_completo.head())

    print("\nInformações sobre o DataFrame completo:")
    df_completo.info()

    # Exibe a quantidade de valores nulos para verificar prontuários que não encontraram correspondência
    print("\nContagem de valores nulos na coluna 'Prontuário' (do df_pacientes_limpo):")
    print(f"Isto indica quantos prontuários de 'df_consolidado' não foram encontrados em 'df_pacientes_limpo'.")
    print(df_completo['Prontuário'].isnull().sum())

else:
    print("Erro: Certifique-se de que as células anteriores que criam os DataFrames 'df_consolidado' e 'df_pacientes_limpo' foram executadas com sucesso.")

In [ ]:
# --- 1. Verificação do DataFrame ---
# Garante que o DataFrame 'df_completo' existe no ambiente do notebook.
if 'df_completo' in locals():

    print(f"Número de linhas ANTES da limpeza: {len(df_completo)}")

    # --- 2. Limpeza dos Dados ---
    # A maneira mais robusta de remover linhas com valores ausentes é usando o método .dropna().
    # Este método remove, por padrão, linhas que contêm valores nulos (NaN - Not a Number).
    # O parâmetro 'subset' especifica que devemos olhar apenas para a coluna 'prontuario_cisam'
    # para decidir se uma linha deve ser removida.
    df_completo_limpo = df_completo.dropna(subset=['prontuario_cisam'])

    # --- Verificação Adicional ---
    # Às vezes, "vazio" pode significar uma string vazia ('') em vez de um valor nulo (NaN).
    # O código abaixo garante que linhas com strings vazias (ou apenas com espaços) também sejam removidas.
    # Primeiro, garantimos que a coluna é do tipo string para usar métodos de string (.str)
    df_completo_limpo['prontuario_cisam'] = df_completo_limpo['prontuario_cisam'].astype(str)
    # Em seguida, mantemos apenas as linhas onde a coluna, sem espaços, não é uma string vazia.
    df_completo_limpo = df_completo_limpo[df_completo_limpo['prontuario_cisam'].str.strip() != '']


    # --- 3. Exibição do Resultado ---
    print(f"Número de linhas APÓS a limpeza: {len(df_completo_limpo)}")

    print("\nVisualizando as primeiras linhas do DataFrame após a remoção:")
    display(df_completo_limpo.head())

    # Opcional: Se você quiser que a variável original 'df_completo' aponte para
    # o dataframe limpo, descomente a linha abaixo.
    # df_completo = df_completo_limpo.copy()
    # print("\nO DataFrame 'df_completo' foi atualizado.")

else:
    print("Erro: O DataFrame 'df_completo' não foi encontrado. Certifique-se de que a célula anterior (que une os dataframes) foi executada.")